**IMPORTING LIBRARIES**

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from textblob import TextBlob
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [5]:
import random

def sentiment_func(label):
    if label <0:
        return "NEGATIVE"
    elif label <= 0.2:
        return "NEUTRAL"
    elif label > 0.2:
        return "POSITIVE"
        

In [7]:
df = pd.read_csv(r'Datafiniti_Hotel_Reviews_Jun19.csv')
df

ParserError: ignored

In [ ]:
df = df.loc[:, df.columns.intersection(['reviews.text'])]
inplace=True
df

In [ ]:
df.info()
df.describe()

In [ ]:
def review_polarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
df['polarity'] = df['reviews.text'].apply(review_polarity)
df.head(10)

In [ ]:
df['sentiment'] = df['polarity'].apply(sentiment_func)
df.head(18)

In [ ]:
rating = df.iloc[:,2]
review = df.iloc[:,0]

In [ ]:
review_train, review_test, rating_train, rating_test = train_test_split(review, rating, test_size=0.2, random_state=42)

In [ ]:
print(rating_train)

In [ ]:
count_vect = CountVectorizer()
# rvwtrain=['review_train']
# rvwtest=['review_test']
# ratetrain=['rating_train']
# ratetest=['rating_test']

X_train_counts = count_vect.fit_transform(review_train)
X_test_counts = count_vect.transform(review_test)
X_train_counts.shape

In [ ]:
# print(X_train_counts[0].toarray())
Index_label = df[df['polarity']==0.2].index.tolist()
  
# Print all the labels
print(Index_label)

In [ ]:
svm_class = svm.SVC(C=0.1, gamma=1, kernel='linear')
svm_class.fit(X_train_counts,rating_train)
# review_test[0]
svmpredict=svm_class.predict(X_test_counts)
print(classification_report(rating_test, svmpredict))

In [ ]:

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train_counts, rating_train)
predictedknn = knn.predict(X_test_counts)
# Calculate the accuracy of the model
print(classification_report(rating_test, predictedknn))

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train_counts, rating_train)
predictdt = dt.predict(X_test_counts)
print(classification_report(rating_test, predictdt))
# max_depth = 5 , criterion = 'entropy', random_state = 42

In [ ]:
log_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs',random_state = 42)
log_classifier.fit(X_train_counts, rating_train)
logpred = log_classifier.predict(X_test_counts)
print(classification_report(rating_test, logpred))

In [ ]:
param_grid = {'C': [0.1, 1], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train_counts, rating_train)
print(grid.best_params_)
print(grid.best_estimator_)